In [1]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [2]:
#! /usr/bin/python3

import sys
import pennylane as qml
import sklearn as skl
import autograd.numpy as np
import itertools
import time

def train_circuit_V2(circuit,n_params,n_qubits,X_train,Y_train,rate_type='accuracy',**kwargs):
    """Develop and train your very own variational quantum classifier.

    Use the provided training data to train your classifier. The code you write
    for this challenge should be completely contained within this function
    between the # QHACK # comment markers. The number of qubits, choice of
    variational ansatz, cost function, and optimization method are all to be
    developed by you in this function.

    Args:
        circuit (qml.QNode): A circuit that you want to train
        X_train (np.ndarray): An array of floats of size (M, n) to be used as training data.
        Y_train (np.ndarray): An array of size (M,) which are the categorical labels
            associated to the training data. The categories are labeled by -1, 0, and 1.
        X_test (np.ndarray): An array of floats of (B, n) to serve as testing data.
        kwargs: hyperparameters for the training (steps, batch_size, learning_rate)

    Returns:
        (p,i,e,w): The number of parameters, the inference time (time to evaluate the accuracy), error rate (accuracy on the test set)
    """

    # Use this array to make a prediction for the labels of the data in X_test
    from autograd.numpy import exp,tanh

    def hinge_loss(labels, predictions,type='L2'):
        loss = 0
        for l, p in zip(labels, predictions):
            if type=='L1':
                loss = loss + np.abs(l - p) # L1 loss
            elif type=='L2':
                loss = loss + (l - p) ** 2 # L2 loss
        loss = loss/len(labels)
        return loss

    def accuracy(labels, predictions):

        loss = 0
        tol = 0.05
        #tol = 0.1
        for l, p in zip(labels, predictions):
            if abs(l - p) < tol:
                loss = loss + 1
        loss = loss / len(labels)

        return loss

    def cost_fcn(params,circuit=None,ang_array=[], actual=[]):
        '''
        use MAE to start
        '''
        labels = {2:-1,1:1,0:0}
        n = len(ang_array[0])
        w = params[-n:]
        theta = params[:-n]
        predictions = [2.*(1.0/(1.0+exp(np.dot(-w,circuit(theta, features=x)))))- 1. for x in ang_array]
        return hinge_loss(actual, predictions)

    var = np.hstack((np.zeros(n_params),5*np.random.random(n_qubits)-2.5))
    steps = kwargs['s']
    batch_size = kwargs['batch_size']
    num_train = len(Y_train)
    validation_size = 5*kwargs['batch_size']
    opt = qml.AdamOptimizer(kwargs['learning_rate'])
    start = time.time()
    for _ in range(steps):
        batch_index = np.random.randint(0, num_train, (batch_size,))
        X_train_batch = X_train[batch_index]
        Y_train_batch = Y_train[batch_index]
        var,cost = opt.step_and_cost(lambda v: cost_fcn(v, circuit,X_train_batch, Y_train_batch), var)
    end = time.time()
    cost_time = (end-start)

    w = var[-n_qubits:]
    theta = var[:-n_qubits]

    if rate_type =='accuracy':
        validation_batch = np.random.randint(0, num_train, (validation_size,))
        X_validation_batch = X_train[validation_batch]
        Y_validation_batch = Y_train[validation_batch]
        start = time.time() # add in timeit function from Wbranch
        predictions=[int(np.round(2.*(1.0/(1.0+exp(np.dot(-w,circuit(theta, features=x)))))- 1.,0)) for x in X_validation_batch]
        end = time.time()
        inftime = (end-start)/len(X_validation_batch)
        err_rate = 1.0 - accuracy(predictions,Y_validation_batch)
    elif rate_type=='batch_cost':
        err_rate = cost
        inftime = cost_time
    # QHACK #
    W_ = np.abs((100.-len(var))/(100.))*np.abs((100.-inftime)/(100.))*(1./err_rate)
    return len(var),inftime,err_rate,W_,var

def train_circuit(circuit,n_params,n_qubits,X_train,Y_train,rate_type='accuracy',**kwargs):
    """Develop and train your very own variational quantum classifier.

    Use the provided training data to train your classifier. The code you write
    for this challenge should be completely contained within this function
    between the # QHACK # comment markers. The number of qubits, choice of
    variational ansatz, cost function, and optimization method are all to be
    developed by you in this function.

    Args:
        circuit (qml.QNode): A circuit that you want to train
        X_train (np.ndarray): An array of floats of size (M, n) to be used as training data.
        Y_train (np.ndarray): An array of size (M,) which are the categorical labels
            associated to the training data. The categories are labeled by -1, 0, and 1.
        X_test (np.ndarray): An array of floats of (B, n) to serve as testing data.
        kwargs: hyperparameters for the training (steps, batch_size, learning_rate)

    Returns:
        (p,i,e,w): The number of parameters, the inference time (time to evaluate the accuracy), error rate (accuracy on the test set)
    """

    # Use this array to make a prediction for the labels of the data in X_test
    from autograd.numpy import exp,tanh

    def hinge_loss(labels, predictions,type='L2'):
        loss = 0
        for l, p in zip(labels, predictions):
            if type=='L1':
                loss = loss + np.abs(l - p) # L1 loss
            elif type=='L2':
                loss = loss + (l - p) ** 2 # L2 loss
        loss = loss/len(labels)
        return loss

    def accuracy(labels, predictions):

        loss = 0
        tol = 0.05
        #tol = 0.1
        for l, p in zip(labels, predictions):
            if abs(l - p) < tol:
                loss = loss + 1
        loss = loss / len(labels)

        return loss

    def cost_fcn(params,circuit=None,ang_array=[], actual=[],r=1):
        '''
        use MAE to start
        '''
        labels = {2:-1,1:1,0:0}
        n = len(ang_array[0])
        w = params[-n*r:]
        theta = params[:-n*r]
        predictions = [2.*(1.0/(1.0+exp(np.dot(-w,circuit(theta, features=x)))))- 1. for x in ang_array]
        return hinge_loss(actual, predictions)

    var = np.hstack((np.zeros(n_params),5*np.random.random(n_qubits)-2.5))
    steps = kwargs['s']
    batch_size = kwargs['batch_size']
    num_train = len(Y_train)
    validation_size = 5*kwargs['batch_size']
    opt = qml.AdamOptimizer(kwargs['learning_rate'])
    start = time.time()
    r=n_qubits//X_train.shape[1]
    for _ in range(steps):
        batch_index = np.random.randint(0, num_train, (batch_size,))
        X_train_batch = X_train[batch_index]
        Y_train_batch = Y_train[batch_index]
        var,cost = opt.step_and_cost(lambda v: cost_fcn(v, circuit,X_train_batch, Y_train_batch,r), var)
        print(_,cost)
    end = time.time()
    cost_time = (end-start)

    w = var[-n_qubits:]
    theta = var[:-n_qubits]

    if rate_type =='accuracy':
        validation_batch = np.random.randint(0, num_train, (validation_size,))
        X_validation_batch = X_train[validation_batch]
        Y_validation_batch = Y_train[validation_batch]
        start = time.time() # add in timeit function from Wbranch
        predictions=[int(np.round(2.*(1.0/(1.0+exp(np.dot(-w,circuit(theta, features=x)))))- 1.,0)) for x in X_validation_batch]
        end = time.time()
        inftime = (end-start)/len(X_validation_batch)
        err_rate = 1.0 - accuracy(predictions,Y_validation_batch) + 10**-6 # add epsilon to avoid division by 0
    elif rate_type=='batch_cost':
        err_rate = cost
        inftime = cost_time
    # QHACK #
    W_ = np.abs((100.-len(var))/(100.))*np.abs((100.-inftime)/(100.))*(1./err_rate)
    #return len(var),inftime,err_rate,W_,var
    return W_,var

def loop_over_hyperparameters(circuit,n_params,X_train,Y_train,batch_sets,learning_rates,**kwargs):
    """
    together with the function train_circuit(...) this executes lines 7-8 in the Algorithm 1 pseudo code of (de Wynter 2020)
    """
    hyperparameter_space = list(itertools.product(batch_sets,learning_rates))
    Wmax = 0.0
    s = kwargs.get('s', None)
    rate_type = kwargs.get('rate_type',None)
    
    for idx,sdx in hyperparameters:
        wtemp,weights=train_circuit(circuit,n_params,X_train,Y_train,s=s,batch_size=idx,rate_type=rate_type,learning_rate=sdx)
        if wtemp>=Wmax:
            Wmax=wtemp
            saved_weights = weights
    return Wmax,saved_weights

def classify_data_DEPRECIATED(X_train,Y_train,X_test,Y_test,**kwargs):
    """Develop and train your very own variational quantum classifier.

    Use the provided training data to train your classifier. The code you write
    for this challenge should be completely contained within this function
    between the # QHACK # comment markers. The number of qubits, choice of
    variational ansatz, cost function, and optimization method are all to be
    developed by you in this function.

    Args:
        X_train (np.ndarray): An array of floats of size (250, 3) to be used as training data.
        Y_train (np.ndarray): An array of size (250,) which are the categorical labels
            associated to the training data. The categories are labeled by -1, 0, and 1.
        X_test (np.ndarray): An array of floats of (50, 3) to serve as testing data.

    Returns:
        str: The predicted categories of X_test, converted from a list of ints to a
            comma-separated string.
    """

    # Use this array to make a prediction for the labels of the data in X_test
    predictions = []

    # QHACK #

    from autograd.numpy import exp,tanh

    def statepreparation(a):
        qml.templates.embeddings.AngleEmbedding(a, wires=range(3), rotation='Y')

    def layer(W):
        qml.templates.layers.BasicEntanglerLayers(W, wires=range(3), rotation=qml.ops.RY)

    def hinge_loss(labels, predictions,type='L2'):
        loss = 0
        for l, p in zip(labels, predictions):
            if type=='L1':
                loss = loss + np.abs(l - p) # L1 loss
            elif type=='L2':
                loss = loss + (l - p) ** 2 # L2 loss
        loss = loss/len(labels)
        return loss

    def accuracy(labels, predictions):

        loss = 0
        tol = 0.05
        #tol = 0.1
        for l, p in zip(labels, predictions):
            if abs(l - p) < tol:
                loss = loss + 1
        loss = loss / len(labels)

        return loss

    def cost_fcn(params,circuit=None,ang_array=[], actual=[]):
        '''
        use MAE to start
        '''
        labels = {2:-1,1:1,0:0}
        w = params[-3:]
        theta = params[:-3]
        predictions = [2.*(1.0/(1.0+exp(np.dot(-w,circuit(theta, angles=x)))))- 1. for x in ang_array]
        return hinge_loss(actual, predictions)

    dev = qml.device("default.qubit", wires=3)
    @qml.qnode(dev)
    def inside_circuit(params,angles=None):
        statepreparation(angles)
        W= np.reshape(params,(len(params)//3,3))
        layer(W)
        return qml.expval(qml.PauliZ(0)),qml.expval(qml.PauliZ(1)),qml.expval(qml.PauliZ(2))


    var = np.hstack((np.zeros(6),5*np.random.random(3)-2.5))
    steps = kwargs['s']
    batch_size = kwargs['batch_size']
    num_train = len(Y_train)
    validation_size = int(num_train//2)
    opt = qml.AdamOptimizer(kwargs['learning_rate'])

    for _ in range(steps):
        batch_index = np.random.randint(0, num_train, (batch_size,))
        X_train_batch = X_train[batch_index]
        Y_train_batch = Y_train[batch_index]
        var,cost = opt.step_and_cost(lambda v: cost_fcn(v, inside_circuit,X_train_batch, Y_train_batch,r), var)

    # need timing values from computing predictions

    
    theta = var[:-3]
    w = var[-3:]
    start = time.time() # add in timeit function from Wbranch
    predictions=[int(np.round(2.*(1.0/(1.0+exp(np.dot(-w,inside_circuit(theta, angles=x)))))- 1.,0)) for x in X_test]
    end = time.time()
    inftime = end-start
    err_rate = 1.0 - accuracy(predictions,Y_test)
    # QHACK #
    W_ = len(var)*inftime*(1./err_rate)
    return len(var),inftime,err_rate,W_

## Import data from sklearn

In [3]:
from sklearn import datasets

In [4]:
n_samples = 100
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=.5,
                                      noise=.05)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=.05)

In [5]:
X_train = noisy_circles[0]
Y_train = noisy_circles[1]
X_test = noisy_circles[0][50:]
Y_test = noisy_circles[1][50:]

### Try running a loop over some hyper_parameters

This uses the circuit classifer built during the Challenge -- the circuit and QNode is constructed inside the function `classify_data`

The following characteristics are hard wired inside the function `classify_data`:
* number of qubits (3)
* number of rotation gates (6)
* the initialization used for the rotations and weights (rotations intialized with 0, weights initialized with random numbers drawn uniformly from $[-2.5,2.5]$
* the rotation gates that are trained (RY)
* the rotation gates used in the `AngleEmbeddding` (RY)
* the optimizer (`AdamOptimizer`)

The following parameters are passed as keywords:
* `s` (the number of steps to train for)
* `batch_size` (the batch size used in training)
* `learning_rate` (the initial learning rate for Adam)

As in (de Wynter 2020) we only train each circuit for a few steps (here I'm using 10).  In (de Wynter 2020) the error rate surrogate is defined using a loss function evaluated over a subset of the data -- here I'm using the accuracy of assigned labels over the test data.  The full number of samples that I generated for the dataset is given by `n_samples` (above).  I've split that data in to train,test sets. 

In [ ]:
batch_sets = [2,4,8]
learning_rates = [0.01,0.05]
hyperparameters = list(itertools.product(batch_sets,learning_rates))
print(hyperparameters)
for idx,sdx in hyperparameters:
    print(idx,sdx,classify_data(X_train,Y_train,X_test,Y_test,s=3,batch_size=idx,learning_rate=sdx))

In [ ]:
#uncommenting and running this cell should cause an error
#inside_circuit 

In [ ]:
X_train.shape[1]

### Try running a loop over some hyper_parameters

This time use a circuit (QNode) that is created outside the function and passed as an argument

Things that are still hard-wired inside the `train_circuit` function:
* Optimizer choice
* Initialization choice (same as above)


In [ ]:
def variational_circuit(params,features=None):
    r_ = QUBITS//len(features)
    large_features = np.tile(features,r_)
    qml.templates.embeddings.AngleEmbedding(large_features, wires=range(QUBITS), rotation='Y') # replace with more general embedding
    if len(params)%QUBITS!=0:
        print('chooose parameter length that is divisible by number of qubits')
        return
    W= np.reshape(params,(len(params)//QUBITS,QUBITS))
    qml.templates.layers.BasicEntanglerLayers(W, wires=range(QUBITS), rotation=qml.ops.RY)
    return [qml.expval(qml.PauliZ(idx)) for idx in range(QUBITS)]


In [ ]:
qubit_sizes=[2,4]
batch_sets = [2,4,8,16,32]
learning_rates = [0.001,0.005,0.01,0.05]
hyperparameters = list(itertools.product(batch_sets,learning_rates))
print(hyperparameters)
results = {}
Wmax = 0.0
for qdx in qubit_sizes:
    QUBITS=qdx
    dev = qml.device("default.qubit",wires=QUBITS)
    # Instantiate the QNode
    outside_circuit = qml.QNode(func=variational_circuit,device=dev)
    n_params=2*QUBITS
    for idx,sdx in hyperparameters:
        p,i,er,wtemp,weights=train_circuit(outside_circuit,n_params,qdx,X_train,Y_train,s=5,batch_size=idx,rate_type='accuracy',learning_rate=sdx)
        print(p,i,er,wtemp,weights)
        if wtemp>=Wmax:
            Wmax=wtemp
            saved_weights = weights
            output = (idx,sdx,p,i,er)
print("Max W coef: ", Wmax)
print("saved weights: ",saved_weights)
print("hyperparameters: ",output[0],output[1])
print("variables (p,i,er): ",p,i,er)

In [ ]:
loop_over_hyperparameters(outside_circuit,4,X_train,Y_train,batch_sets=[2,4,8,16,32],learning_rates=[0.001,0.005,0.01,0.05],s=5,rate_type='accuracy')

### Pull in more detailed circuit design

In [ ]:
import sys
import pennylane as qml
import sklearn as skl
import autograd.numpy as np
import itertools
import time

In [6]:


def zz_layer(wires, params):
    nq = len(wires)
    for n in range(nq - 1):
        zz_gate([n, n + 1], params[n])
    zz_gate([nq - 1, 0], params[nq - 1])


def zz_gate(wires, gamma):
    if isinstance(gamma,(float,qml.variable.Variable)):
        qml.CNOT(wires=wires)
        qml.RZ(gamma, wires=wires[1])
        qml.CNOT(wires=wires)
    else:
        qml.CNOT(wires=wires)
        qml.RZ(gamma._value, wires=wires[1])
        qml.CNOT(wires=wires)

def CNOT_layer(wires):
    nq = len(wires)
    for n in range(nq-1):
        qml.CNOT(wires=[n,n+1])
    qml.CNOT(wires=[nq-1,0])

def x_layer(wires, params):
    nqubits = len(wires)
    if isinstance(params,(list,np.ndarray,qml.variable.Variable)):
        for n in range(nqubits):
            qml.RX(params[n], wires=[n, ])
    else:
        for n in range(nqubits):
            qml.RX(params[n]._value, wires=[n, ])


def y_layer(wires, params):
    nqubits = len(wires)
    if isinstance(params,(list,np.ndarray,qml.variable.Variable)):
        for n in range(nqubits):
            qml.RY(params[n], wires=[n, ])
    else:
        for n in range(nqubits):
            qml.RY(params[n]._value, wires=[n, ])

#TODO: ADD W-COST HERE

string_to_layer_mapping = {'ZZ': zz_layer, 'X': x_layer, 'Y': y_layer}

In [7]:

def string2circuit(gate_params=[],features=[],arch_params=[],n_wires=None):
    string_to_param_count = {'E1':0,'ZZ':n_wires,'X':n_wires,'Y':n_wires}
    pdx = 0
    for gdx in arch_params:
        if gdx=='E1':
            r_ = n_wires//len(features)
            large_features = np.tile(features,r_)
            qml.templates.embeddings.AngleEmbedding(large_features, wires=range(n_wires), rotation='Y') # replace with more general embedding
            #qml.templates.embeddings.AngleEmbedding(features, wires=range(n_wires), rotation='Y') # replace with more general embedding
        elif gdx=='ZZ':
            #w = gate_params[pdx:pdx+n_wires]
            #zz_layer(range(n_wires),w)
            CNOT_layer(range(n_wires))
            #pdx+=n_wires
        elif gdx=='X':
            w = gate_params[pdx:pdx+n_wires]
            x_layer(range(n_wires),w)
            pdx+=n_wires
        elif gdx=='Y':
            w = gate_params[pdx:pdx+n_wires]
            y_layer(range(n_wires),w)
            pdx+=n_wires
    return [qml.expval(qml.PauliZ(idx)) for idx in range(n_wires)]
    

In [13]:
qubits = [2,4]
batch_sets = [2,4,8,16,32]
learning_rates = [0.001,0.005,0.01,0.05]
hyperparameters = list(itertools.product(batch_sets,learning_rates))
print(hyperparameters)

Wmax = 0.0


ARCH = ['E1','ZZ','Y','ZZ','Y','ZZ','Y','ZZ','Y','ZZ','Y']
for WIRES in qubits:
    def temp_circuit(w,features=None): return string2circuit(gate_params=w,features=features,arch_params=ARCH,n_wires=WIRES)
    dev = qml.device("default.qubit",wires=WIRES)
    outside_circuit = qml.QNode(temp_circuit,device=dev)

    #if using zz-layer, use this line
    #n_params=np.sum([WIRES for x in ARCH if (x!='E1')])
    #if useing CNOT-layer, use this line
    n_params=np.sum([WIRES for x in ARCH if (x=='X') or (x=='Y')])
    print(n_params)
    for idx,sdx in hyperparameters:
        wtemp,weights=train_circuit(outside_circuit,n_params,WIRES,X_train,Y_train,s=5,batch_size=idx,rate_type='batch_cost',learning_rate=sdx)
        print('final weights:',weights)
        print(wtemp)
        if wtemp>=Wmax:
            Wmax=wtemp
            saved_weights = weights
            saved_qubits = qdx
    print("Max W coef: ", Wmax)
    print("saved weights: ",saved_weights)
    print("number of qubits: ",saved_qubits)


[(2, 0.001), (2, 0.005), (2, 0.01), (2, 0.05), (4, 0.001), (4, 0.005), (4, 0.01), (4, 0.05), (8, 0.001), (8, 0.005), (8, 0.01), (8, 0.05), (16, 0.001), (16, 0.005), (16, 0.01), (16, 0.05), (32, 0.001), (32, 0.005), (32, 0.01), (32, 0.05)]
10
0 0.13752273400732473
1 0.1927262072942658
2 0.2878470875055854
3 0.21109554010146525
4 0.10298211283225189
final weights: [ 0.00169041 -0.00270752 -0.00100336 -0.00295001 -0.00267867 -0.00208673
  0.00251852 -0.00270768 -0.00100259 -0.00295052  0.64650615  0.46085116]
8.48897500158883
0 0.3567203317478812
1 0.018604066645504083
2 0.35217352240216976
3 0.3422041323253365
4 0.6738125291825741
final weights: [ 0.00638273  0.01410457  0.01519151 -0.01529088  0.01409094  0.01506289
  0.00578289  0.01410562  0.01519714 -0.01533379  0.15833052  0.25851329]
1.297454808929577
0 0.19272582415664935
1 0.17345870548721962
2 0.2625015411978689
3 0.2878416762870544
4 0.131249061586121
final weights: [-1.34772010e-03 -3.18868454e-02 -3.06771405e-02  3.12186920e-

3 1.9045439128166224
4 1.3748105400963446
final weights: [ 3.51255048e-03  1.42197946e-03  3.50238609e-03 -3.20196042e-03
  3.27564331e-03  3.38104273e-03 -1.43448675e-03 -2.97366915e-03
 -2.62548217e-03 -1.90015396e-03  3.48734173e-03 -2.62779997e-03
  2.91290826e-03  2.14301248e-03  2.94924224e-03  4.86605401e-03
  3.45368182e-03  3.50494778e-03 -3.20577041e-03  4.64542825e-03
 -2.10371399e-02 -7.17287881e-01  1.21775864e+00 -2.14736171e+00]
0.5228686504094435
0 0.3195892531535749
1 0.40380270173221755
2 0.32212487574462334
3 0.2949472824965064
4 0.4845788928410664
final weights: [ 0.01578626  0.01636465 -0.01449126 -0.02008728  0.01628477 -0.01662682
 -0.00384255  0.01329518  0.01012396 -0.00663425  0.01619006 -0.01287872
  0.01526652 -0.01347198  0.01563209 -0.01608487 -0.01532038 -0.01454695
 -0.02009254  0.01836382  2.24427152  0.93218082 -0.90401962 -1.74934669]
1.4833359408837181
0 0.27536259855030265
1 0.19670528316641275
2 0.1656983000788766
3 0.4507829476243292
4 0.146615745

In [16]:
for idx, sdx in list(itertools.product([6],[0.1])):
    print(idx,sdx)

6 0.1
